In [ ]:
#importing libraries
import pandas as pd
import nltk
import gensim
import time
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import re
import string
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from gensim import corpora,models

In [ ]:
!pip install -U mittens

In [ ]:
from mittens import GloVe

In [ ]:
# from glove import Glove
# from glove import Corpus

In [ ]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
# from glove import Glove, Corpus

In [ ]:
from scipy.spatial import distance
import nltk
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
from gensim import corpora,models
import time
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from gensim.models import Word2Vec,FastText
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#reading the pre-downloaded csv file from Bugzilla
data=pd.read_csv('preproccessed_whole_dataset.csv')
data

In [ ]:
data = data.drop(columns = ["Unnamed: 0"])
data.tail()

In [ ]:
data.reset_index(drop=True)

In [ ]:
data =  data.rename(columns={"Bug_Id": "Bug ID"})
data

In [ ]:
#Dropping duplicate entities
data = data.drop_duplicates()
data = data.dropna(axis=0, subset=['Bug ID'])
data

## On complete data

In [ ]:
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [ ]:
#Text Cleaning round 1 (removing punctutions)

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*\f\w*', '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*]\)', '', text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# data = data.drop(columns=['Unnamed: 0'])
data = data.dropna(axis=0, subset=['Description'])

In [ ]:
data['Description'] = data['Description'].apply(clean_text_round1)

In [ ]:
# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
data['Description'] = data['Description'].apply(clean_text_round2)

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
#Lemmatization
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [ ]:
data['Description'] = data['Description'].map(preprocess)

In [ ]:
data['Duplicate_Bug_Ids'].isnull().sum()

In [ ]:
#Saving all the duplicate reports into a csv file as a testing set
duplicate_reports = data.dropna(axis=0, subset=['Duplicate_Bug_Ids'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('duplicate_reports.csv')

In [ ]:
#Seperating all the master reports into a dataframe
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

In [ ]:
master_reports.Description.isna().sum()

In [ ]:
mr = pd.DataFrame()
count = 0

for i in range(len(master_reports)):
    if len(master_reports.Description.iloc[i]) > 2:
        mr = pd.concat([mr, master_reports.iloc[[i]]], ignore_index=True)
        count += 1

In [ ]:
master_reports.to_csv('master_reports.csv')

In [ ]:
#importing the CSV file of the master reports from google drive into a dataframe
master_reports = pd.read_csv('master_reports.csv')
master_reports = master_reports.drop(columns=['Unnamed: 0'])

In [ ]:
master_reports['Description'] = master_reports['Description'].map(preprocess)

In [ ]:
#Creating a dictionary using gensim library
dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
#Print top 20 words from the dictionary
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

In [ ]:
#Creating BoW using the the dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]

In [ ]:
#Printing the BoW for single document
bow_doc_8 = bow_corpus[8]
for i in range(len(bow_doc_8)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_8[i][0],
                                               dictionary[bow_doc_8[i][0]],
bow_doc_8[i][1]))

In [ ]:
# open a file, where you stored the pickled data
f= open('bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bow_corpus, f)

In [ ]:
#open a file, where you stored the pickled data
file = open('dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [ ]:
#Preparing the parameters for LDA model
corpus = bow_corpus
no_of_topics = 10
dictionary = dictionary
p = 20
k = 2
epochs = 100

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=no_of_topics, id2word=dictionary, passes=p, workers=k, iterations=epochs)

In [ ]:
# save model to disk (no need to use pickle module)
lda_model.save('lda_model.model')

In [ ]:
# Printing the topics and the propability distributions of words in those topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
#Let's evaluate the model using Perplexity and Coherence Bag of words- Title

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# for visualization of LDA
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
vis

In [ ]:
type(dictionary)

In [ ]:
for c in range(10):
    exec(f'topic_{c} = pd.DataFrame()')
    for i in range(len(master_reports)):
        topic = lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic = np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]), 0]) == c:
            exec(f'topic_{c} = pd.concat([topic_{c}, master_reports.loc[[i]]])')
            exec(f'topic_{c} = topic_{c}.reset_index(drop=True)')
            exec(f'topic_{c}.to_csv("topic_{c}.csv")')

In [ ]:
# open a file, where you stored the pickled data
# f = open('/content/drive/MyDrive/BugEnricher/bow_corpus.pickle', 'rb')
# bow_corpus=pickle.load(f)

# file = open('/content/drive/MyDrive/BugEnricher/bow_corpus.pickle', 'rb')
# dictionary=pickle.load(file)

# later on, load trained model from file
# lda_model =  models.LdaModel.load('/content/drive/MyDrive/BugEnricher/LDA/lda_model.model')

In [ ]:
#Import all the clusters from the drive
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    # exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [ ]:
#creating a corpus for Word2Vec and FastText models
for i in range(10):
    exec('sent_{} = []'.format(i))
    exec('x= topic_{}'.format(i))
    for j in range(len(x)):
        exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

In [ ]:
for sent in range(10):
    exec('print(len(sent_{}))'.format(sent))

In [ ]:
!pip install glove-python3

In [ ]:
from glove import Glove, Corpus

In [ ]:
#Training GloVe model for each cluster
for cluster in range(10):
    vector_size = 100
    exec('glove_corpus{}=Corpus()'.format(cluster, cluster))
    exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
    exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
    exec('glove{}.fit(glove_corpus{}.matrix, epochs=100, no_threads=3, verbose=True)'.format(cluster, cluster))
    exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=100,ignore_missing=False)'.format(cluster, cluster))
    exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

    #Save the all the models in individual file
    exec('path = get_tmpfile("glove{}.model")'.format(cluster))
    exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

## Classification

In [ ]:
# importing all the clusters created using LDA based topic modeling
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    # exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [ ]:
#import the duplicate reports for testing purpose
test = pd.read_csv('/content/drive/MyDrive/BugEnricher/duplicates/Firefox_duplicate_reports.csv')
# test = test.drop(columns=['Unnamed: 0'])

In [ ]:
# preprocessing textually similar and dissimilar duplicates to evaluate
def lemmatize2(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize2(token))
    return result

#import the duplicate reports for testing purpose
test_sim = pd.read_csv('sim.csv')
# test_sim = test_sim.drop(columns=['Unnamed: 0'])

#import the duplicate reports for testing purpose
test_dissim = pd.read_csv('dis.csv')
# test_dissim = test_dissim.drop(columns=['Unnamed: 0'])

test_sim['Description']= test_sim['Description'].fillna('').astype(str).map(preprocess2)
test_dissim['Description']= test_dissim['Description'].fillna('').astype(str).map(preprocess2)

In [ ]:
def lemmatize2(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [ ]:
test['Description']= test['Description'].fillna('').astype(str).map(preprocess2)

In [ ]:
for mod in range(10):
    #import all the trained GloVe models
    exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [ ]:
#This will return the index of cluster in which the master report of duplicate report may reside
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= lda_model[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    return sim

In [ ]:
# Convert multiple word embeddings into a single document vector by averaging the word embeddings by GloVe model

def average_word_vectors_glove(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[model.wv.key_to_index[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector


def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.wv.key_to_index)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
    return np.array(features)

In [ ]:
# Similarity between two feature vectors using the average of cosine similarity and Euclidean similarity
def sim(vec1, vec2):
    # Ensure vec1 and vec2 are 2-D arrays for cosine similarity
    vec1 = vec1.reshape(1, -1) if len(vec1.shape) == 1 else vec1
    vec2 = vec2.reshape(1, -1) if len(vec2.shape) == 1 else vec2

    sim1 = 1 / (1 + distance.euclidean(vec1.flatten(), vec2.flatten()))  # Euclidean similarity
    sim2 = cosine_similarity(vec1, vec2)[0][0]  # Cosine similarity
    sim = (sim1 + sim2) / 2

    return sim


In [ ]:
# Different kinds of fusion of two master report feature vectors and two duplicate report feature vectors
def fusion(vec1, vec2, vec3, vec4, fusion_no):

    # fusion_no = 1 : concatenation of the vectors
    if (fusion_no == '1'):
        master = np.concatenate((vec1, vec2), axis=1)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        return vec_duplicate, master

    #fusion_no = 2 : average of the vectors
    elif (fusion_no == '2'):
        vec3 = [vec3]
        vec4 = [vec4]
        avg1 = (np.add(vec1, vec2))/2
        avg2 = (np.add(vec3, vec4))/2
        return avg2, avg1

    #fusion_no = 3 : Dimensionality reduction using PCA on concatenation of the vectors
    elif (fusion_no == '3'):
        master = np.concatenate((vec1, vec2), axis=1)
        pca = PCA(n_components=100)
        avg_fit = pca.fit(master)
        master = pca.transform(master)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master

    #fusion_no = 3 : Dimensionality reduction using PCA on average of the vectors
    elif (fusion_no == '4'):
        vec3 = [vec3]
        vec4 = [vec4]
        avg1 = (np.add(vec1, vec2))/2
        pca = PCA(n_components=100)
        avg_fit = pca.fit(avg1)
        master = pca.transform(avg1)
        avg2 = (np.add(vec3, vec4))/2
        vec_duplicate = pca.transform(avg2)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [ ]:
# creation of feature vectors by multimodality feature extraction
def feature_vectors_multi_modality(DR, corpus, model1, model2, fusion_no):
    master_glove2 = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)
    vec_duplicate2 = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)
    vec_duplicate , master= fusion(master_glove2,vec_duplicate2, fusion_no)
    return vec_duplicate,master

In [ ]:
# creation of feature vectors by singlemodality feature extraction
def feature_vectors_single_modality(DR, corpus, model1):
    master = averaged_word_vectorizer_glove(corpus=sent, model=model1, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove(corpus=DR, model=model1, num_features=100)

    vec_duplicate = [vec_duplicate]

    return vec_duplicate, master

In [ ]:
test = test.rename(columns={'Bug ID':'Bug_ID'})

In [ ]:
def compare_topn(model1, cluster, sent, DR, topn, modal):
    similarity=[]
    if (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model1)

    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)
    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1][1:]
    return(max_similar_reports)

## LDA and GLOVE

## on testset

In [ ]:
#Returns Top-N master reports
dup_id = test.Duplicate_Bug_Ids

def compare_topn2(model1, cluster, sent, DR, topn, modal):
    similarity = []

    if modal == 'single':
        vec_duplicate, master = feature_vectors_single_modality2(DR, sent, model1)
    else:
        raise ValueError('Invalid Modality entered')

    for vec_master in master:
        unified_sim = sim(vec_duplicate, vec_master)  # Flattening done in sim function
        similarity.append(unified_sim)

    # Convert similarity list to a numpy array for sorting
    similarity = np.array(similarity)
    max_similar_reports = similarity.argsort()[-topn:][::-1]

    return max_similar_reports


def feature_vectors_single_modality2(DR, corpus, model1):
    master = averaged_word_vectorizer_glove2(corpus=corpus, model=model1, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove2(corpus=[DR], model=model1, num_features=100)

    return vec_duplicate.flatten(), [vec.flatten() for vec in master]


In [ ]:
def average_word_vectors_glove2(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector
def averaged_word_vectorizer_glove2(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove2(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove2(corpus, model, vocabulary, num_features)
    return np.array(features)

In [ ]:
#top1
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

In [ ]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

In [ ]:
#top5
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i]
    n = 5
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 5
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")

In [ ]:
# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

In [ ]:
#top10
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i]
    n = 10
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 10
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")

# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

In [ ]:
#top100
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i]
    n = 100
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 100
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")

# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

# On Textually Similar and Dissimilar datasets

In [ ]:
nan_value = float("NaN")
test_sim.replace("", nan_value, inplace=True)
test_sim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)

In [ ]:
nan_value = float("NaN")
test_dissim.replace("", nan_value, inplace=True)
test_dissim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)

In [ ]:
cond = (test_sim['Duplicate_Bug_Ids'] == 'NaN')
cond.unique()

In [ ]:
cond = (test_dissim['Duplicate_Bug_Ids'] == 'NaN')
cond.unique()

In [ ]:
test_sim = test_sim.reset_index(drop=True)

In [ ]:
test_dissim = test_dissim.reset_index(drop=True)

# Textually Similar

In [ ]:
#top 1
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test_sim.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 1
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 1
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")


# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))


In [ ]:
#top 5
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test_sim.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 5
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 5
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")


# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))


In [ ]:
#top 10
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 10
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 10                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")


# Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

In [ ]:
#top 100
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test_sim.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 100
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 100
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")


# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

# Textually dissimilar

In [ ]:
#top 1
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test_dissim.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i]
    n = 1
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 1
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")


# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

In [ ]:
#top 5
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 5
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 5                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

In [ ]:
#top 10
vec_acc = []
t1 = time.time()
no_of_test_samples = min(len(test_sim.Description), 200)

for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i]
    n = 10
    max_cluster = sim_with_clusters_lda_topn(sample, n)
    v = []

    for max in max_cluster:
        cluster = eval(f'topic_{max}')
        model1 = eval(f'glove{max}')
        sent = cluster.Description
        cluster = cluster.rename(columns={'Bug ID': 'Bug_ID'})

        topn = 10
        fusion_no = '4'
        modal = 'single'

        max_sim = compare_topn2(model1, cluster, sent, sample, topn, modal)

        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")
    vec_acc.append("0" if all(x == v[0] for x in v) else "1")


# Evaluating the performance by Recall Rate
total_sum = 0
for num in vec_acc:
    total_sum += int(num)

recall_rate = (total_sum / len(vec_acc)) * 100 if len(vec_acc) > 0 else 0

print("Recall Rate: {:.2f} %".format(recall_rate))

In [ ]:
#top 100
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 100
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 100
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")